In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
#https://www.kaggle.com/hiralmshah/reduce-memory-usage-trick-with-elo-merchant-data
def reduce_memory(dataframe, verbose =True):
    numerical = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    initial_memory = dataframe.memory_usage().sum()/1024**2
    for col in dataframe.columns:
        col_type = dataframe[col].dtypes
        if col_type == object:
            dataframe[col] = dataframe[col].astype('category')
        if col_type in numerical:
            c_min = dataframe[col].min()
            c_max = dataframe[col].max()
            if str(col_type)[:3] =='int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    dataframe[col] = dataframe[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    dataframe[col] = dataframe[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    dataframe[col] = dataframe[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    dataframe[col] = dataframe[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    dataframe[col] = dataframe[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    dataframe[col] = dataframe[col].astype(np.float32)
                elif c_min > np.finfo(np.float64).min and c_max < np.finfo(np.float64).max:
                    dataframe[col] = dataframe[col].astype(np.float64)
    final_memory = dataframe.memory_usage().sum()/ 1024**2
    if verbose:
        print('Memory usage reduced to {:5.2f} Mb({:.1f}% reduction)'.format(final_memory, (initial_memory-final_memory)/initial_memory*100))
    return dataframe

### Feature engineering 

In [3]:
historical_transactions = reduce_memory(pd.read_csv('historical_transactions.csv'))
historical_transactions.head()

Memory usage reduced to 1622.97 Mb(47.8% reduction)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [4]:
new_merchant_trans = reduce_memory(pd.read_csv('new_merchant_transactions.csv'))
new_merchant_trans.head()

Memory usage reduced to 169.08 Mb(19.4% reduction)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551270,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671875,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659668,2018-03-22 21:07:53,NaN,-1,29


In [72]:
historical_transactions.isna().sum()/ len(historical_transactions)*100

authorized_flag         0.000000
card_id                 0.000000
city_id                 0.000000
category_1              0.000000
installments            0.000000
category_3              0.611970
merchant_category_id    0.000000
merchant_id             0.475678
month_lag               0.000000
purchase_amount         0.000000
purchase_date           0.000000
category_2              9.112500
state_id                0.000000
subsector_id            0.000000
dtype: float64

In [73]:
new_merchant_trans.isna().sum()/ len(new_merchant_trans)*100

authorized_flag         0.000000
card_id                 0.000000
city_id                 0.000000
category_1              0.000000
installments            0.000000
category_3              2.848758
merchant_category_id    0.000000
merchant_id             1.335486
month_lag               0.000000
purchase_amount         0.000000
purchase_date           0.000000
category_2              5.692473
state_id                0.000000
subsector_id            0.000000
dtype: float64

We need to impute the missing values.

In [41]:
# imputing missing values in historical_transactions
historical_transactions['category_3'].fillna(historical_transactions['category_3'].mode()[0], inplace = True)
historical_transactions['merchant_id'].fillna(historical_transactions['merchant_id'].mode()[0], inplace = True)
historical_transactions['category_2'].fillna(historical_transactions['category_2'].mode()[0], inplace = True)

#encoding categorical features
historical_transactions['category_1'] = historical_transactions['category_1'].map({'Y':1, 'N':0}).astype(int)
historical_transactions['category_3'] =historical_transactions['category_3'].map({'A':0, 'B':1, 'C':2}).astype(int)
historical_transactions['authorized_flag'] = historical_transactions['authorized_flag'].map({'Y':1, 'N':0}).astype(int)

In [42]:
# imputing missing values in new_merchant_transactions
new_merchant_trans['category_3'].fillna(new_merchant_trans['category_3'].mode()[0], inplace = True)
new_merchant_trans['merchant_id'].fillna(new_merchant_trans['merchant_id'].mode()[0], inplace = True)
new_merchant_trans['category_2'].fillna(new_merchant_trans['category_2'].mode()[0], inplace = True)

#encoding categorical features
new_merchant_trans['category_1'] = new_merchant_trans['category_1'].map({'Y':1, 'N':0}).astype(int)
new_merchant_trans['category_3'] = new_merchant_trans['category_3'].map({'A':0, 'B':1, 'C':2}).astype(int)
new_merchant_trans['authorized_flag'] = new_merchant_trans['authorized_flag'].map({'Y':1, 'N':0}).astype(int)

In [43]:
historical_transactions.dtypes

authorized_flag            int64
card_id                 category
city_id                    int16
category_1                 int64
installments               int16
category_3                 int64
merchant_category_id       int16
merchant_id             category
month_lag                   int8
purchase_amount          float32
purchase_date           category
category_2               float16
state_id                    int8
subsector_id                int8
dtype: object

 https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/72993
While going through kaggle discussions it is understood that 999 and -1 values in installments are missing values. So we need to replace them with Nan.

In [44]:
historical_transactions['installments'].replace(-1, np.nan, inplace = True)
historical_transactions['installments'].replace(999, np.nan, inplace = True)
new_merchant_trans['installments'].replace(-1, np.nan, inplace = True)
new_merchant_trans['installments'].replace(999, np.nan, inplace = True)

In [45]:
historical_transactions['installments'].fillna(historical_transactions['installments'].mode()[0], inplace = True)
new_merchant_trans['installments'].fillna(new_merchant_trans['installments'].mode()[0], inplace = True)

#### Feature engineering on date

In [46]:
def add_date_part(df):
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['purchase_year'] = df['purchase_date'].dt.year
    df['purchase_day'] = df['purchase_date'].dt.day
    df['purchase_month'] = df['purchase_date'].dt.month
    df['purchase_week'] = df['purchase_date'].dt.week
    df['purchase_day'] = df['purchase_date'].dt.day
    df['purchase_dayofweek'] = df['purchase_date'].dt.dayofweek
    df['purchase_dayofyear'] = df['purchase_date'].dt.dayofyear
    df['purchase_hour'] = df['purchase_date'].dt.hour
    df['purchase_minute'] = df['purchase_date'].dt.minute
    df['purchase_second'] = df['purchase_date'].dt.second
    df['purchase_month_end'] = df['purchase_date'].dt.is_month_end.astype(int)
    df['purchase_month_start'] = df['purchase_date'].dt.is_month_start.astype(int)
    df['purchase_quarter_start'] = df['purchase_date'].dt.is_quarter_start.astype(int)
    df['purchase_quarter_end'] = df['purchase_date'].dt.is_quarter_end.astype(int)
    df['purchase_year_start'] = df['purchase_date'].dt.is_year_start.astype(int)
    df['purchase_year_end'] = df['purchase_date'].dt.is_year_end.astype(int)
    return df

In [47]:
historical_transactions = add_date_part(historical_transactions)
new_merchant_trans = add_date_part(new_merchant_trans)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [48]:
def add_extra_date_features(df):
    df['purchase_is_weekend'] = (df.purchase_dayofweek >=5).astype(int)
    df['purchase_is_weekday'] = (df.purchase_dayofweek <5).astype(int)
    df['month_difference'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_difference'] += df['month_lag']
    return df

In [49]:
historical_transactions = add_extra_date_features(historical_transactions)
new_merchant_trans = add_extra_date_features(new_merchant_trans)

In [50]:
#denormalizing purchase_amount
#https://www.kaggle.com/raddar/towards-de-anonymizing-the-data-some-insights
historical_transactions['purchase_amount'] = np.round(historical_transactions['purchase_amount']/0.00150265118 + 497.06 , 2)

In [51]:
new_merchant_trans['purchase_amount'] = np.round(new_merchant_trans['purchase_amount']/0.00150265118 + 497.06, 2)

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:2219: RuntimeWarning: overflow encountered in multiply
  result = self._values.round(decimals)


In [141]:
historical_transactions.head().T

,0,1,2,3,4
index,0,1,2,3,4
authorized_flag,1,1,1,1,1
card_id,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc
city_id,88,88,88,88,88
category_1,0,0,0,0,0
installments,0,0,0,0,0
category_3,0,0,0,0,0
merchant_category_id,80,367,80,560,80
merchant_id,M_ID_e020e9b302,M_ID_86ec983688,M_ID_979ed661fc,M_ID_e6d5ae8ea6,M_ID_e020e9b302
month_lag,-8,-7,-6,-5,-11


#### Aggregating features

In [52]:
#aggregating historical_transactions by card_id
aggregation_dict = {'card_id' : ['size'],
                    'city_id': ['nunique'],
                   'category_1': ['max', 'min', 'sum', 'mean'],
                   'installments' : ['sum', 'max', 'min', 'mean', 'var','skew'],
                   'category_3' : ['sum', 'mean'],
                   'merchant_category_id': ['nunique'],
                   'merchant_id': ['nunique'],
                   'month_lag': ['sum', 'max', 'min', 'var', 'mean', 'skew'],
                   'purchase_amount' : ['sum', 'mean', 'var', 'max', 'min', 'skew'],
                   'purchase_date' : ['max', 'min'],
                   'category_2': ['sum', 'mean'],
                   'state_id' : ['nunique'],
                   'subsector_id' : ['nunique'],
                   'purchase_year' : ['max', 'min', 'nunique'],
                   'purchase_day' : ['max', 'min', 'nunique'],
                   'purchase_month' : ['max', 'min', 'nunique'],
                   'purchase_week' : ['max', 'min', 'nunique'],
                   'purchase_dayofweek' : ['max', 'min', 'nunique','mean'],
                   'purchase_dayofyear' : ['max', 'min', 'nunique', 'mean'],
                   'purchase_hour' : ['max', 'min', 'nunique', 'mean'],
                   'purchase_minute': ['max', 'min', 'nunique', 'mean'],
                   'purchase_second': ['max', 'min', 'nunique', 'mean'],
                   'purchase_month_end' : ['nunique', 'mean'],
                   'purchase_month_start' : ['nunique', 'mean'],
                   'purchase_quarter_start' : ['nunique', 'mean'],
                   'purchase_quarter_end' : ['nunique', 'mean'],
                   'purchase_year_start': ['nunique', 'mean'],
                   'purchase_year_end': ['nunique', 'mean'],
                   'purchase_is_weekday' : ['nunique', 'mean'],
                   'purchase_is_weekend' : ['nunique', 'mean'],
                   'month_difference' : ['max', 'min', 'mean', 'var', 'skew']}



In [53]:
historical_transactions_aggregated = historical_transactions.groupby('card_id').agg(aggregation_dict)

In [54]:
historical_transactions_aggregated.columns = ['trans_' + '_'.join(col) for col in historical_transactions_aggregated.columns.values]

In [55]:
historical_transactions_aggregated.reset_index(inplace = True)

In [56]:
historical_transactions_aggregated['trans_purchase_max_min'] = (historical_transactions_aggregated['trans_purchase_date_max'] - historical_transactions_aggregated['trans_purchase_date_min']).dt.days
historical_transactions_aggregated['trans_purchase_date_uptomax'] = (datetime.datetime.today() - historical_transactions_aggregated['trans_purchase_date_max']).dt.days
historical_transactions_aggregated['trans_purchase_date_uptomin'] = (datetime.datetime.today() - historical_transactions_aggregated['trans_purchase_date_min']).dt.days

In [57]:
historical_transactions_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325540 entries, 0 to 325539
Data columns (total 91 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   card_id                               325540 non-null  category      
 1   trans_card_id_size                    325540 non-null  int64         
 2   trans_city_id_nunique                 325540 non-null  int64         
 3   trans_category_1_max                  325540 non-null  int64         
 4   trans_category_1_min                  325540 non-null  int64         
 5   trans_category_1_sum                  325540 non-null  int64         
 6   trans_category_1_mean                 325540 non-null  float64       
 7   trans_installments_sum                325540 non-null  float64       
 8   trans_installments_max                325540 non-null  float64       
 9   trans_installments_min                325540 non-null  floa

In [58]:
new_merchant_trans_aggregated =  new_merchant_trans.groupby('card_id').agg(aggregation_dict)

In [59]:
new_merchant_trans_aggregated.columns = ['new_' + '_'.join(col) for col in new_merchant_trans_aggregated.columns.values]

In [60]:
new_merchant_trans_aggregated.reset_index(inplace = True)

In [61]:
new_merchant_trans_aggregated['new_purchase_max_min'] = (new_merchant_trans_aggregated['new_purchase_date_max'] - new_merchant_trans_aggregated['new_purchase_date_min']).dt.days
new_merchant_trans_aggregated['new_purchase_date_uptomax'] = (datetime.datetime.today() - new_merchant_trans_aggregated['new_purchase_date_max']).dt.days
new_merchant_trans_aggregated['new_purchase_date_uptomin'] = (datetime.datetime.today() - new_merchant_trans_aggregated['new_purchase_date_min']).dt.days

In [62]:
new_merchant_trans_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290001 entries, 0 to 290000
Data columns (total 91 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   card_id                             290001 non-null  category      
 1   new_card_id_size                    290001 non-null  int64         
 2   new_city_id_nunique                 290001 non-null  int64         
 3   new_category_1_max                  290001 non-null  int64         
 4   new_category_1_min                  290001 non-null  int64         
 5   new_category_1_sum                  290001 non-null  int64         
 6   new_category_1_mean                 290001 non-null  float64       
 7   new_installments_sum                290001 non-null  float64       
 8   new_installments_max                290001 non-null  float64       
 9   new_installments_min                290001 non-null  float64       
 10  new_inst

In [63]:
historical_transactions_aggregated.to_csv(r'historical_transactions_aggregated.csv', index = False)
new_merchant_trans_aggregated.to_csv(r'new_merchant_trans_aggregated.csv', index = False)

---

### Train and test data

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [120]:
train.head().T

,0,1,2,3,4
first_active_month,2017-06,2017-01,2016-08,2017-09,2017-11
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_1,5,4,2,4,1
feature_2,2,1,2,3,3
feature_3,1,0,0,0,0
target,-0.820283,0.392913,0.688056,0.142495,-0.159749


In [121]:
train.isna().sum()

first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
target                0
dtype: int64

In [122]:
test.isna().sum()

first_active_month    1
card_id               0
feature_1             0
feature_2             0
feature_3             0
dtype: int64

In [4]:
test['first_active_month'].fillna(test['first_active_month'].mode()[0], inplace = True)

In [124]:
train.dtypes

first_active_month     object
card_id                object
feature_1               int64
feature_2               int64
feature_3               int64
target                float64
dtype: object

In [125]:
test.dtypes

first_active_month    object
card_id               object
feature_1              int64
feature_2              int64
feature_3              int64
dtype: object

In [5]:
train['first_active_month'] = pd.to_datetime(train['first_active_month'])
test['first_active_month'] = pd.to_datetime(test['first_active_month'])

In [6]:
train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] =1

In [7]:
for feature in ['feature_1', 'feature_2', 'feature_3']:
    mean_value = train.groupby(feature)['target'].mean()
    train[feature] = train[feature].map(mean_value)
    test[feature] = test[feature].map(mean_value)

In [8]:
train['date_elapsed'] = (datetime.datetime.today() - train['first_active_month']).dt.days
train['first_month'] = train['first_active_month'].dt.month
train['first_year'] = train['first_active_month'].dt.year
train['first_week'] = train['first_active_month'].dt.dayofweek
train['first_quarter'] = train['first_active_month'].dt.quarter

In [130]:
train['date_elapsed_feature_1'] = train['date_elapsed'] * train['feature_1']
train['date_elapsed_feature_2'] = train['date_elapsed'] * train['feature_2']
train['date_elapsed_feature_3'] = train['date_elapsed'] * train['feature_3']

train['date_elapsed_feature_1_ratio'] = train['date_elapsed'] / train['feature_1']
train['date_elapsed_feature_2_ratio'] = train['date_elapsed'] / train['feature_2']
train['date_elapsed_feature_3_ratio'] = train['date_elapsed'] / train['feature_3']

train['feature_sum'] = train['feature_1'] + train['feature_2'] + train['feature_3']
train['feature_mean'] = train[['feature_1', 'feature_2', 'feature_3']].mean(axis = 1)
train['feature_max'] = train[['feature_1', 'feature_2', 'feature_3']].max(axis = 1)
train['feature_min'] = train[['feature_1', 'feature_2', 'feature_3']].min(axis = 1)
train['feature_var'] = train[['feature_1', 'feature_2', 'feature_3']].std(axis = 1)

In [131]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,outliers,date_elapsed,first_month,first_year,...,date_elapsed_feature_2,date_elapsed_feature_3,date_elapsed_feature_1_ratio,date_elapsed_feature_2_ratio,date_elapsed_feature_3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var
0,2017-06-01,C_ID_92a2005557,-0.499879,-0.332899,-0.421057,-0.820283,0,1349,6,2017,...,-449.081074,-568.005338,-2698.651998,-4052.277205,-3203.844893,-1.253835,-0.417945,-0.332899,-0.499879,0.083533
1,2017-01-01,C_ID_3d0044924f,-0.404652,-0.398675,-0.357939,0.392913,0,1500,1,2017,...,-598.012005,-536.908315,-3706.888148,-3762.466273,-4190.659628,-1.161266,-0.387089,-0.357939,-0.404652,0.025421
2,2016-08-01,C_ID_d639edf6cd,-0.350452,-0.332899,-0.357939,0.688056,0,1653,8,2016,...,-550.282443,-591.672963,-4716.769558,-4965.466435,-4618.106910,-1.041290,-0.347097,-0.332899,-0.357939,0.012853
3,2017-09-01,C_ID_186d6a6901,-0.404652,-0.501889,-0.357939,0.142495,0,1257,9,2017,...,-630.875048,-449.929168,-3106.372268,-2504.535574,-3511.772768,-1.264480,-0.421493,-0.357939,-0.501889,0.073438
4,2017-11-01,C_ID_cdbd2c0db2,-0.315476,-0.501889,-0.357939,-0.159749,0,1196,11,2017,...,-600.259791,-428.094897,-3791.102086,-2382.994866,-3341.352610,-1.175304,-0.391768,-0.315476,-0.501889,0.097703


In [132]:
test['date_elapsed'] = (datetime.datetime.today() - test['first_active_month']).dt.days
test['first_month'] = test['first_active_month'].dt.month
test['first_year'] = test['first_active_month'].dt.year
test['first_week'] = test['first_active_month'].dt.dayofweek
test['first_quarter'] = test['first_active_month'].dt.quarter

In [133]:
test['date_elapsed_feature_1'] = test['date_elapsed'] * test['feature_1']
test['date_elapsed_feature_2'] = test['date_elapsed'] * test['feature_2']
test['date_elapsed_feature_3'] = test['date_elapsed'] * test['feature_3']

test['date_elapsed_feature_1_ratio'] = test['date_elapsed'] / test['feature_1']
test['date_elapsed_feature_2_ratio'] = test['date_elapsed'] / test['feature_2']
test['date_elapsed_feature_3_ratio'] = test['date_elapsed'] / test['feature_3']

test['feature_sum'] = test['feature_1'] + test['feature_2'] + test['feature_3']
test['feature_mean'] = test[['feature_1', 'feature_2', 'feature_3']].mean(axis = 1)
test['feature_max'] = test[['feature_1', 'feature_2', 'feature_3']].max(axis = 1)
test['feature_min'] = test[['feature_1', 'feature_2', 'feature_3']].min(axis = 1)
test['feature_var'] = test[['feature_1', 'feature_2', 'feature_3']].std(axis = 1)

In [134]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,date_elapsed,first_month,first_year,first_week,first_quarter,...,date_elapsed_feature_2,date_elapsed_feature_3,date_elapsed_feature_1_ratio,date_elapsed_feature_2_ratio,date_elapsed_feature_3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var
0,2017-04-01,C_ID_0ab67a22ab,-0.377533,-0.501889,-0.421057,1410,4,2017,5,2,...,-707.664135,-593.689790,-3734.773878,-2809.383579,-3348.718531,-1.300479,-0.433493,-0.377533,-0.501889,0.063104
1,2017-01-01,C_ID_130fd0cbdd,-0.350452,-0.501889,-0.357939,1500,1,2017,6,1,...,-752.834186,-536.908315,-4280.190161,-2988.705935,-4190.659628,-1.210280,-0.403427,-0.350452,-0.501889,0.085353
2,2017-08-01,C_ID_b709037bc5,-0.499879,-0.398675,-0.421057,1288,8,2017,1,3,...,-513.492975,-542.320886,-2576.622515,-3230.704373,-3058.971254,-1.319610,-0.439870,-0.398675,-0.499879,0.053161
3,2017-12-01,C_ID_d27d835a9f,-0.350452,-0.398675,-0.357939,1166,12,2017,4,4,...,-464.854665,-417.356730,-3327.134485,-2924.690449,-3257.539417,-1.107065,-0.369022,-0.350452,-0.398675,0.025952
4,2015-12-01,C_ID_2b5e3df5c2,-0.499879,-0.398675,-0.421057,1897,12,2015,1,4,...,-756.285849,-798.744348,-3794.916857,-4758.265680,-4505.332662,-1.319610,-0.439870,-0.398675,-0.499879,0.053161


---

#### Merging aggregated historical and new transactions with train and test

In [159]:
hist_aggregated = pd.read_csv('historical_transactions_aggregated.csv')
new_aggregated = pd.read_csv('new_merchant_trans_aggregated.csv')

In [136]:
hist_aggregated.head()

,card_id,trans_card_id_size,trans_city_id_nunique,trans_category_1_max,trans_category_1_min,trans_category_1_sum,trans_category_1_mean,trans_installments_sum,trans_installments_max,trans_installments_min,...,trans_purchase_is_weekend_nunique,trans_purchase_is_weekend_mean,trans_month_difference_max,trans_month_difference_min,trans_month_difference_mean,trans_month_difference_var,trans_month_difference_skew,trans_purchase_max_min,trans_purchase_date_uptomax,trans_purchase_date_uptomin
0,C_ID_00007093c1,149,4,1,0,28,0.187919,192.0,6.0,1.0,...,2,0.167785,36,35,35.912752,0.080174,-2.955089,377,1078,1455
1,C_ID_0001238066,123,18,1,0,2,0.016260,201.0,10.0,0.0,...,2,0.422764,36,35,35.934959,0.061309,-3.571387,151,1077,1229
2,C_ID_0001506ef0,66,3,0,0,0,0.000000,1.0,1.0,0.0,...,2,0.484848,36,35,35.969697,0.029837,-5.608353,398,1087,1486
3,C_ID_0001793786,216,10,1,0,2,0.009259,5.0,1.0,0.0,...,2,0.171296,40,39,39.935185,0.060896,-3.560004,283,1196,1480
4,C_ID_000183fdda,144,9,1,0,4,0.027778,268.0,10.0,0.0,...,2,0.229167,36,35,35.847222,0.130342,-1.950606,202,1079,1282


In [137]:
new_aggregated.head()

,card_id,new_card_id_size,new_city_id_nunique,new_category_1_max,new_category_1_min,new_category_1_sum,new_category_1_mean,new_installments_sum,new_installments_max,new_installments_min,...,new_purchase_is_weekend_nunique,new_purchase_is_weekend_mean,new_month_difference_max,new_month_difference_min,new_month_difference_mean,new_month_difference_var,new_month_difference_skew,new_purchase_max_min,new_purchase_date_uptomax,new_purchase_date_uptomin
0,C_ID_00007093c1,2,2,0,0,0,0.000000,2.0,1.0,1.0,...,1,0.000000,36,36,36.000000,0.000000,NaN,6,1036,1043
1,C_ID_0001238066,26,8,1,0,2,0.076923,43.0,10.0,0.0,...,2,0.461538,36,35,35.807692,0.161538,-1.658711,60,1015,1075
2,C_ID_0001506ef0,2,1,0,0,0,0.000000,0.0,0.0,0.0,...,1,0.000000,36,36,36.000000,0.000000,NaN,5,1055,1060
3,C_ID_0001793786,31,7,0,0,0,0.000000,0.0,0.0,0.0,...,2,0.451613,40,39,39.709677,0.212903,-0.971526,46,1135,1181
4,C_ID_000183fdda,11,2,0,0,0,0.000000,17.0,4.0,0.0,...,2,0.181818,36,35,35.818182,0.163636,-1.922718,59,1015,1075


In [160]:
train_hist = pd.merge(train, hist_aggregated, how = 'left', on = 'card_id')
test_hist = pd.merge(test, hist_aggregated, how = 'left', on = 'card_id')

In [161]:
train_hist.shape

(201917, 113)

In [140]:
test_hist.shape

(123623, 111)

In [162]:
train_trans = pd.merge(train_hist, new_aggregated, how = 'left', on = 'card_id')
test_trans = pd.merge(test_hist, new_aggregated, how = 'left', on = 'card_id')

In [163]:
print(train_trans.shape, test_trans.shape)

(201917, 203) (123623, 201)


In [173]:
train_trans['trans_purchase_date_max'] = pd.to_datetime(train_trans['trans_purchase_date_max'])
train_trans['trans_purchase_date_min'] = pd.to_datetime(train_trans['trans_purchase_date_min'])
train_trans['new_purchase_date_max'] = pd.to_datetime(train_trans['new_purchase_date_max'])
train_trans['new_purchase_date_min'] = pd.to_datetime(train_trans['new_purchase_date_min'])

test_trans['trans_purchase_date_max'] = pd.to_datetime(test_trans['trans_purchase_date_max'])
test_trans['trans_purchase_date_min'] = pd.to_datetime(test_trans['trans_purchase_date_min'])
test_trans['new_purchase_date_max'] = pd.to_datetime(test_trans['new_purchase_date_max'])
test_trans['new_purchase_date_min'] = pd.to_datetime(test_trans['new_purchase_date_min'])

In [174]:
#https://www.kaggle.com/chauhuynh/my-first-kernel-3-699
train_trans['trans_first_buy'] = train_trans['trans_purchase_date_min'] - train_trans['first_active_month']
train_trans['new_first_buy'] = train_trans['new_purchase_date_min'] - train_trans['first_active_month']

train_trans['card_id_total'] = train_trans['trans_card_id_size'] + train_trans['new_card_id_size']
train_trans['card_id_ratio'] = train_trans['trans_card_id_size'] / train_trans['new_card_id_size']

train_trans['purchase_amount_total'] = train_trans['trans_purchase_amount_sum'] + train_trans['new_purchase_amount_sum']
train_trans['purchase_amount_mean'] = train_trans['trans_purchase_amount_mean'] + train_trans['new_purchase_amount_mean']
train_trans['purchase_amount_var'] = train_trans['trans_purchase_amount_var'] + train_trans['new_purchase_amount_var']
train_trans['purchase_amount_max'] = train_trans['trans_purchase_amount_max'] + train_trans['new_purchase_amount_max']
train_trans['purchase_amount_min'] = train_trans['trans_purchase_amount_min'] + train_trans['new_purchase_amount_min']
train_trans['purchase_amount_skew'] = train_trans['trans_purchase_amount_skew'] + train_trans['new_purchase_amount_skew']

train_trans['installments_total'] = train_trans['trans_installments_sum'] + train_trans['new_installments_sum']
train_trans['installments_max'] = train_trans['trans_installments_max'] + train_trans['new_installments_max']
train_trans['installments_min'] = train_trans['trans_installments_min'] + train_trans['new_installments_min']
train_trans['installments_mean'] = train_trans['trans_installments_mean'] + train_trans['new_installments_mean']
train_trans['installments_var'] = train_trans['trans_installments_var'] + train_trans['new_installments_var']
train_trans['installments_skew'] = train_trans['trans_installments_skew'] + train_trans['new_installments_skew']

train_trans['month_lag_total'] = train_trans['trans_month_lag_sum'] + train_trans['new_month_lag_sum']
train_trans['month_lag_max'] = train_trans['trans_month_lag_max'] + train_trans['new_month_lag_max']
train_trans['month_lag_min'] = train_trans['trans_month_lag_min'] + train_trans['new_month_lag_min']
train_trans['month_lag_mean'] = train_trans['trans_month_lag_mean'] + train_trans['new_month_lag_mean']
train_trans['month_lag_var'] = train_trans['trans_month_lag_var'] + train_trans['new_month_lag_var']
train_trans['month_lag_skew'] = train_trans['trans_month_lag_skew'] + train_trans['new_month_lag_skew']

train_trans['month_diff_max'] = train_trans['trans_month_difference_max'] + train_trans['new_month_difference_max']
train_trans['month_diff_min'] = train_trans['trans_month_difference_min'] + train_trans['new_month_difference_min']
train_trans['month_diff_mean'] = train_trans['trans_month_difference_mean'] + train_trans['new_month_difference_mean']
train_trans['month_diff_var'] = train_trans['trans_month_difference_var'] + train_trans['new_month_difference_var']
train_trans['month_diff_skew'] = train_trans['trans_month_difference_skew'] + train_trans['new_month_difference_skew']



In [175]:
train_trans.dtypes

first_active_month    datetime64[ns]
card_id                       object
feature_1                    float64
feature_2                    float64
feature_3                    float64
                           ...      
month_diff_max               float64
month_diff_min               float64
month_diff_mean              float64
month_diff_var               float64
month_diff_skew              float64
Length: 230, dtype: object

In [176]:
test_trans['trans_first_buy'] = test_trans['trans_purchase_date_min'] - test_trans['first_active_month']
test_trans['new_first_buy'] = test_trans['new_purchase_date_min'] - test_trans['first_active_month']

test_trans['card_id_total'] = test_trans['trans_card_id_size'] + test_trans['new_card_id_size']
test_trans['card_id_ratio'] = test_trans['trans_card_id_size'] / test_trans['new_card_id_size']

test_trans['purchase_amount_total'] = test_trans['trans_purchase_amount_sum'] + test_trans['new_purchase_amount_sum']
test_trans['purchase_amount_mean'] = test_trans['trans_purchase_amount_mean'] + test_trans['new_purchase_amount_mean']
test_trans['purchase_amount_var'] = test_trans['trans_purchase_amount_var'] + test_trans['new_purchase_amount_var']
test_trans['purchase_amount_max'] = test_trans['trans_purchase_amount_max'] + test_trans['new_purchase_amount_max']
test_trans['purchase_amount_min'] = test_trans['trans_purchase_amount_min'] + test_trans['new_purchase_amount_min']
test_trans['purchase_amount_skew'] = test_trans['trans_purchase_amount_skew'] + test_trans['new_purchase_amount_skew']

test_trans['installments_total'] = test_trans['trans_installments_sum'] + test_trans['new_installments_sum']
test_trans['installments_max'] = test_trans['trans_installments_max'] + test_trans['new_installments_max']
test_trans['installments_min'] = test_trans['trans_installments_min'] + test_trans['new_installments_min']
test_trans['installments_mean'] = test_trans['trans_installments_mean'] + test_trans['new_installments_mean']
test_trans['installments_var'] = test_trans['trans_installments_var'] + test_trans['new_installments_var']
test_trans['installments_skew'] = test_trans['trans_installments_skew'] + test_trans['new_installments_skew']

test_trans['month_lag_total'] = test_trans['trans_month_lag_sum'] + test_trans['new_month_lag_sum']
test_trans['month_lag_max'] = test_trans['trans_month_lag_max'] + test_trans['new_month_lag_max']
test_trans['month_lag_min'] = test_trans['trans_month_lag_min'] + test_trans['new_month_lag_min']
test_trans['month_lag_mean'] = test_trans['trans_month_lag_mean'] + test_trans['new_month_lag_mean']
test_trans['month_lag_var'] = test_trans['trans_month_lag_var'] + test_trans['new_month_lag_var']
test_trans['month_lag_skew'] = test_trans['trans_month_lag_skew'] + test_trans['new_month_lag_skew']

test_trans['month_diff_max'] = test_trans['trans_month_difference_max'] + test_trans['new_month_difference_max']
test_trans['month_diff_min'] = test_trans['trans_month_difference_min'] + test_trans['new_month_difference_min']
test_trans['month_diff_mean'] = test_trans['trans_month_difference_mean'] + test_trans['new_month_difference_mean']
test_trans['month_diff_var'] = test_trans['trans_month_difference_var'] + test_trans['new_month_difference_var']
test_trans['month_diff_skew'] = test_trans['trans_month_difference_skew'] + test_trans['new_month_difference_skew']



In [177]:
len(train_trans.columns[train_trans.isna().any()])

120

In [178]:
train_trans.replace([-np.inf, np.inf], np.nan, inplace = True)
test_trans.replace([-np.inf, np.inf], np.nan, inplace = True)

In [179]:
train_na = train_trans.columns[train_trans.isna().any()]
test_na = test_trans.columns[test_trans.isna().any()]

In [180]:
print(len(train_na), len(test_na))

120 120


In [181]:
for i in range(len(train_na)):
    train_trans[train_na[i]].fillna(train_trans[train_na[i]].mode()[0], inplace = True)
    test_trans[train_na[i]].fillna(test_trans[train_na[i]].mode()[0], inplace = True)

In [182]:
train_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201917 entries, 0 to 201916
Columns: 230 entries, first_active_month to month_diff_skew
dtypes: datetime64[ns](5), float64(164), int64(58), object(1), timedelta64[ns](2)
memory usage: 365.9+ MB


In [183]:
s = train_trans.select_dtypes(include = ['datetime64[ns]']).columns

In [184]:
train_trans = train_trans.drop(s, axis = 1)
test_trans = test_trans.drop(s, axis = 1)

In [185]:
ns = train_trans.select_dtypes(include = ['timedelta64[ns]']).columns

In [187]:
for n in ns:
    train_trans[n] = train_trans[n].astype(int)
    test_trans[n] = test_trans[n].astype(int)

In [188]:
train_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201917 entries, 0 to 201916
Columns: 225 entries, card_id to month_diff_skew
dtypes: float64(164), int64(60), object(1)
memory usage: 358.2+ MB


In [190]:
test_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123623 entries, 0 to 123622
Columns: 223 entries, card_id to month_diff_skew
dtypes: float64(163), int64(59), object(1)
memory usage: 211.3+ MB


In [193]:
print(train_trans.shape, test_trans.shape)

(201917, 225) (123623, 223)


In [194]:
train_trans.to_csv(r'train_trans.csv', index = False)
test_trans.to_csv(r'test_trans.csv', index = False)

___

In [3]:
train_trans = reduce_memory(pd.read_csv('train_trans.csv'))

Memory usage reduced to 95.50 Mb(72.4% reduction)


In [5]:
train_trans.to_pickle('final_train.pkl')

In [6]:
test_trans = reduce_memory(pd.read_csv('test_trans.csv'))

Memory usage reduced to 56.99 Mb(72.9% reduction)


In [7]:
test_trans.to_pickle('final_test.pkl')

In [8]:
train_trans.dtypes

card_id            category
feature_1           float16
feature_2           float16
feature_3           float16
target              float16
                     ...   
month_diff_max      float16
month_diff_min      float16
month_diff_mean     float16
month_diff_var      float16
month_diff_skew     float16
Length: 225, dtype: object